# Task B1 - Face Shape Detector

by Mohamed Azmil Irfan Mohamed Amiruddin

(AMLS 19/20 ASSIGNMENT)

## Import Library

In [13]:
import os
import cv2
import dlib
import numpy as np
import pandas as pd
from keras.preprocessing import image


%matplotlib inline
# required magic function

import sys
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.
C:\Users\Bear-\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bear-\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bear-\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bear-\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

## Directory List
relevant folder
1. Image directory : ..\AMLS_19-20_SN15053755\dataset\cartoon_set\img
2. Label CSV file directory : ..\AMLS_19-20_SN15053755\dataset\cartoon_set\labels.csv

In [3]:
os.chdir("..")
print(os.path.abspath(os.curdir))

C:\Users\Bear-\Desktop\AMLS_19-20_SN15053755


In [4]:
labels_filename = 'labels.csv'

basedir = os.path.abspath(os.curdir)
dataset_dir = os.path.join(basedir,'dataset')
cartoon_dir = os.path.join(dataset_dir,'cartoon_set')
images_dir = os.path.join(cartoon_dir,'img')
labels_dir = os.path.join(cartoon_dir,labels_filename)

## Feature Extraction

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [14]:
def extract_features_labels(images_dir, labels_dir): ## eye colour, face shape
    """ return:
        landmark_features:  an array containing 68 landmark points for each image in celeba folder
        fshape_labels:      an array containing the face shape label (0,1,2,3,4) for each image
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)] ##filename in matrix
    target_size = None
    labels_file = open(labels_dir, 'r')
    lines = labels_file.readlines()
    fshape_labels = {line.split('\t')[0] : int(line.split('\t')[2]) for line in lines[1:]} ##fetch filename + faceshape
#     ecolor_labels = {line.split('\t')[0] : int(line.split('\t')[1]) for line in lines[1:]} 
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        error_features = []
        for img_path in image_paths:
            file_name= img_path.split('.')[0].split('\\')[-1] ##getting name of file; remove png/jpg + dir

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(fshape_labels[file_name])
            if features is None:
                error_features.append(file_name)
                

    landmark_features = np.array(all_features)
    fshape_labels = np.array(all_labels)
    return landmark_features, fshape_labels, error_features

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image
    
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

In [15]:
feat , lab , err = extract_features_labels(images_dir, labels_dir)

In [20]:
print(len(lab))
print(len(err))

7815
2185


In [21]:
## Need to find other way to extract the inputs coz the error margin quite huge

In [22]:
print(err)

['1', '10', '100', '1005', '1011', '1016', '1018', '102', '1020', '1036', '1042', '1043', '1048', '1049', '1056', '1068', '1082', '1092', '1094', '1095', '1098', '11', '1100', '1104', '1108', '1109', '1110', '1122', '1128', '1132', '1134', '1139', '114', '1141', '1144', '1147', '115', '1154', '1156', '1159', '1163', '1164', '117', '1174', '1175', '1177', '1178', '1181', '1190', '1191', '1192', '1198', '1199', '12', '1201', '1204', '1209', '1211', '1217', '1218', '1231', '1260', '1262', '1264', '1267', '1269', '1272', '1274', '1278', '1283', '1293', '1294', '1295', '1299', '1302', '1303', '1311', '1313', '1318', '1326', '1329', '1334', '1336', '1337', '1340', '1341', '1342', '1356', '1362', '1366', '137', '1371', '1374', '1376', '1377', '1386', '1388', '1389', '1390', '1394', '1400', '1403', '1414', '1416', '1418', '1425', '1426', '143', '1436', '1443', '1445', '1459', '1462', '1463', '1469', '1474', '1477', '1481', '1483', '1487', '1491', '1494', '1499', '1515', '1518', '152', '1524', 

In [ ]:
cv2